In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

m_A = pd.read_csv('413_12x.csv')  
m_B = pd.read_csv('913_16x.csv')  
m_C = pd.read_csv('913_8x.csv')  

data_1_size = len(m_A)
m_size = 25     ####### 测试多少个月 #######
buy = 1     ##### 多 ###################
sell = 0     ##### 空 ####################
rrr = 0.25     ###### 系数 ###################
m = 1000     ###### 总资金 ###################

In [2]:
###### 两个模型组合

m_A['final'] = 0
condition = (m_A['low'] == m_B['low'])
m_A.loc[condition, 'final'] = m_A.loc[condition, 'volume']
m_A.loc[~condition, 'low'] = 2

In [2]:
###### 三个模型组合

m_A['final'] = 0
condition = (m_A['low'] == m_B['low']) & (m_B['low'] == m_C['low'])
m_A.loc[condition, 'final'] = m_A.loc[condition, 'volume']
m_A.loc[~condition, 'low'] = 2

In [3]:
#######   可视化   ###########################################################################

data_1_new = m_A
data_1_new['high'] = data_1_new['final'].cumsum()
data_1_new['open'] = rrr * data_1_new['high'] / m   

##############################################################################################

### 胜率
wp_win = data_1_new['final'] > 0
wp_lost = data_1_new['final'] < 0
wp_nothing = data_1_new['final'] == 0

wp_win_a = wp_win.sum()            
wp_lost_a = wp_lost.sum()
wp_nothing_a = wp_nothing.sum()

### 盈亏比
rrr_win = data_1_new.loc[wp_win, 'final'].sum()
rrr_lost = data_1_new.loc[wp_lost, 'final'].sum()

##############################################################################################

###### 计算回撤数据
data_1_new['cum_max_open'] = data_1_new['open'].cummax()  
data_1_new['down'] = data_1_new['open'] - data_1_new['cum_max_open']  
data_1_new['down'] = data_1_new['down'].clip(upper=0)  

##############################################################################################

###### 计算回撤面积
downarea = data_1_new['down'].sum()

##############################################################################################

###### 计算真实收益率

data_1_new['re'] = data_1_new['close'].pct_change() * 100

data_1_new['re_real'] = np.where(
    data_1_new['low'] == 0, 
    -data_1_new['re'], 
    data_1_new['re'])

data_1_new['re_real'] = np.where(
    data_1_new['low'] == 2, 
    0,
    data_1_new['re_real'])

##############################################################################################

###### 计算夏普比率和索提诺比率
re_real = data_1_new['re_real'][1:]  
mean_re = re_real.mean()
std_re = re_real.std()
sharpe = round(mean_re / std_re * 100 if std_re != 0 else 0, 4)

neg_re = re_real[re_real < 0]
std_neg_re = neg_re.std() if not neg_re.empty else 0
sortino = round(mean_re / std_neg_re * 100 if std_neg_re != 0 else 0, 4)

##############################################################################################

data_1_new.to_csv('./final.csv',index = False)

s = np.argmax((np.maximum.accumulate(data_1_new['open']) - data_1_new['open'])) 
if s == 0:
    e = 0
else:
    e = np.argmax(data_1_new['open'][:s])  
maxdrawdown = data_1_new['open'][e] - data_1_new['open'][s] # 最大回撤
drawdown_days = s - e # 回撤持续周期数

start_DAY = data_1_new.index[s] #开始回撤的日期
end_DAY = data_1_new.index[e] #结束回撤的日期
start_net_value = data_1_new[data_1_new.index == start_DAY]['open'].values[0] #开始回撤的净值
end_net_value = data_1_new[data_1_new.index == end_DAY]['open'].values[0] #结束回撤的净值
fig=plt.figure(figsize=(20,11))  
plt.plot(data_1_new['eob'], data_1_new['open'])
plt.plot([start_DAY, end_DAY], [start_net_value, end_net_value], linestyle='--', color='r')

plt.xticks(range(0,data_1_size,int(data_1_size/m_size))) 

plt.legend(['All:' + str(round(data_1_new['open'].iloc[-1]*100,2)) + '%' +
            '   ' + str(m_size) + 'm'
            '   Year:'+ str(round(data_1_new['open'].iloc[-1]/m_size*100*12,2)) + '%' +
            '   CalmarY:'+ str(round((data_1_new['open'].iloc[-1]/m_size*100*12)/(maxdrawdown*100),2)) +
            '   WP:' + str(round(wp_win_a/(wp_win_a + wp_lost_a)*100,2)) + '%' +
            '   RRR:' + str(round(rrr_win/(rrr_win+abs(rrr_lost))*100,2)) + '%' + ' / ' + str(round(rrr_win/abs(rrr_lost),2)) +
            '   T/N:' + str(wp_win_a + wp_lost_a ) + ' / ' + str(wp_nothing_a) +
            '   Sharpe:' + str(sharpe) +
            '   Sortino:' + str(sortino),

            'MD:'+ str(round(maxdrawdown*100,2)) + '%' +
            '   DA:'+ str(round(downarea,4)) + '%' +
            '   MDT:' + str(drawdown_days)+
            '   Date:' + str(data_1_new['eob'].iloc[e]) + ' - ' + str(data_1_new['eob'].iloc[s])] ,

            loc='upper left',fontsize = 11)   ########### 默认是10


plt.plot(data_1_new['eob'], data_1_new['down'], color='#ec700a')   ### 桔色
plt.fill_between(data_1_new['eob'], data_1_new['down'], 0, where=(data_1_new['down']<0), facecolor='#FF0000', alpha=0.1)   
plt.xticks(range(0,data_1_size,int(data_1_size/m_size)))                                           ### 红色 + 透明度

fig.autofmt_xdate()
plt.grid(1)
plt.savefig("final.jpg")
plt.close()
